### Dataset from ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED

In [1]:
import datetime
import geopandas
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

#from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib

 # to enable ipympl interactive interface for plots
%matplotlib widget

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages

In [2]:
import time
import matplotlib
from distinctipy import distinctipy # generate N distinct colors

In [3]:
#local scripts
from scripts import veg_indices, utilities, plots
#from scripts.utilities import *

In [4]:
BANDS_DICT = {   'B2': 'Blue',
            'B3': 'Green',
            'B4': 'Red',
            'B5': 'Red_Edge_1',
            'B6': 'Red_Edge_2',
            'B7': 'Red_Edge_3',
            'B8': 'NIR',
            'B8A': 'Red_Edge_4',
            'B11': 'SWIR_1',
            'B12': 'SWIR_2'}

BANDS = list(BANDS_DICT.keys())

In [5]:
NUMERIC_COLS = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12','RVI',
                'ARVI','PSSRa','NDI45','GNDVI','MCARI','IRECI','CIr','MTCI',
                'NDVIre','NIRv','EVI','NDTI','NDMI','MSI','GCI','NBRI','BSI',
                'NDWI','NDSI','NDVI','RVI_diff','ARVI_diff','PSSRa_diff',
                'NDI45_diff','GNDVI_diff','MCARI_diff','IRECI_diff','CIr_diff',
                'MTCI_diff','NDVIre_diff','NIRv_diff','EVI_diff','NDTI_diff',
                'NDMI_diff','MSI_diff','GCI_diff','NBRI_diff','BSI_diff','NDWI_diff',
                'NDSI_diff','NDVI_diff','RVI_prev','ARVI_prev','PSSRa_prev','NDI45_prev',
                'GNDVI_prev','MCARI_prev','IRECI_prev','CIr_prev','MTCI_prev','NDVIre_prev',
                'NIRv_prev','EVI_prev','NDTI_prev','NDMI_prev','MSI_prev','GCI_prev',
                'NBRI_prev','BSI_prev','NDWI_prev','NDSI_prev','NDVI_prev']

NUM_SAMPLES = 16

In [6]:
file_name = "merged_images_groundTruth"
df = geopandas.read_file(f'../data/{file_name}')

# For each 3-week image, standarize each column, after removing outliers
df_trimmed = utilities.get_rm_outlier_standarize(df, NUMERIC_COLS, rm_outliers=True)

In [7]:

print(df_trimmed.columns, df_trimmed.shape)

Index(['level_0', 'INDEX', 'index', 'B11', 'B12', 'B2', 'B3', 'B4', 'B5', 'B6',
       'B7', 'B8', 'B8A', 'NDVI', 'finHarvDat', 'lat', 'lon', 'point_idx',
       'start_date', 'end_date', 'har_evnt', 'image_idx', 'RVI', 'ARVI',
       'PSSRa', 'NDI45', 'GNDVI', 'MCARI', 'IRECI', 'CIr', 'MTCI', 'NDVIre',
       'NIRv', 'EVI', 'NDTI', 'NDMI', 'MSI', 'GCI', 'NBRI', 'BSI', 'NDWI',
       'NDSI', 'RVI_prev', 'ARVI_prev', 'PSSRa_prev', 'NDI45_prev',
       'GNDVI_prev', 'MCARI_prev', 'IRECI_prev', 'CIr_prev', 'MTCI_prev',
       'NDVIre_prev', 'NIRv_prev', 'EVI_prev', 'NDTI_prev', 'NDMI_prev',
       'MSI_prev', 'GCI_prev', 'NBRI_prev', 'BSI_prev', 'NDWI_prev',
       'NDSI_prev', 'NDVI_prev', 'RVI_diff', 'ARVI_diff', 'PSSRa_diff',
       'NDI45_diff', 'GNDVI_diff', 'MCARI_diff', 'IRECI_diff', 'CIr_diff',
       'MTCI_diff', 'NDVIre_diff', 'NIRv_diff', 'EVI_diff', 'NDTI_diff',
       'NDMI_diff', 'MSI_diff', 'GCI_diff', 'NBRI_diff', 'BSI_diff',
       'NDWI_diff', 'NDSI_diff', 'NDVI_diff', '

# NOTE: df_trimmed has NaN because we set outliers to NaN

In [ ]:
stretched_df_trimmed = utilities.stretch_cols(df_trimmed, NUMERIC_COLS)
stretched_df_trimmed

### Producing Box Plots per Numeric Column

In [ ]:
%%script echo skipping

def save_multi_image(filename):
    pp = PdfPages(filename)
    for sampleIdx in stretched_df_trimmed.image_idx.unique():
        curr_df = stretched_df_trimmed[stretched_df_trimmed.image_idx == sampleIdx]
        fig = plt.figure(figsize=(16,int(len(NUMERIC_COLS) * 1.5)))            # set showfliers to False to remove outliers
        sns.boxplot(data=curr_df, x="value", y="class", hue="har_evnt", showfliers = True).set(title= ("Sample Index:", sampleIdx))
        fig.savefig(pp, format='pdf')
        plt.close() # closing figure
    pp.close()

save_multi_image("../plots/box_plots/everything_trimmed_standarized.pdf")

### Producing Separability Plots

In [ ]:
%%script echo skipping

def plot_lines_Mean_95CI(df: pd.DataFrame):
    sns.set(rc={'figure.figsize':(len(BANDS) * 8, len(BANDS) * 5)})

    fig, axes = plt.subplots(len(BANDS), 1)
    fig.suptitle('Mean and 95% CI by Band')
    fig.subplots_adjust(hspace=0.5, wspace=0.5)


    for i in range(len(BANDS)):
        band_name = BANDS[i]
        sns.lineplot(ax=axes[i], x="finHarvDat", y=band_name,
            hue="har_evnt",
            data=df)
        sns.scatterplot(ax=axes[i], data=df, x="finHarvDat", y=band_name, hue="har_evnt", style="har_evnt", alpha=0.5)

        ax2 = axes[i].twinx()
        sns.histplot(ax=ax2, data=df, x="finHarvDat", bins=40, element="step", fill=False, color='green')
        if(i % 2 == 0):
            #axes[i].tick_params(bottom=False)
            #axes[i].legend_.remove()
            pass
        title = (BANDS_DICT[band_name] + f' ({band_name})')
        axes[i].set_title(title)
    plt.close() # closing figure


    utilities.saveFigsAsPDF([fig], "../plots/line_plots/mean&CIByBand.pdf")

plot_lines_Mean_95CI(df.dropna(subset=["finHarvDat"]))

In [ ]:
%%script echo skipping

def plot_lines_Median_IQR(df: pd.DataFrame):

    sns.set(rc={'figure.figsize':(len(BANDS) * 8, len(BANDS) * 5)})
    fig, axes = plt.subplots(len(BANDS), 1)
    fig.suptitle('Median and IQR by Band')
    fig.subplots_adjust(hspace=0.5, wspace=0.5)

    # https://stackoverflow.com/questions/52525476/seaborn-lineplot-using-median-instead-of-mean
    for i in range(len(BANDS)):
        band_name = BANDS[i]
        sns.lineplot(ax=axes[i], x="finHarvDat", y=band_name,
            hue="har_evnt", estimator="median",errorbar=("pi", 50),# show inner quartile range #https://seaborn.pydata.org/tutorial/error_bars.html
            data=df)
        
        sns.scatterplot(ax=axes[i], data=df, x="finHarvDat", y=band_name, hue="har_evnt", style="har_evnt", alpha=0.5)

        ax2 = axes[i].twinx()
        sns.histplot(ax=ax2, data=df, x="finHarvDat", bins=40, element="step", fill=False, color='green')

        if(i % 2 == 0):
            #axes[i].tick_params(bottom=False)
            #axes[i].legend_.remove()
            pass

        title = (BANDS_DICT[band_name] + f' ({band_name})')
        axes[i].set_title(title)
    plt.close() # closing figure
    utilities.saveFigsAsPDF([fig], "../plots/line_plots/median&IQRByBand.pdf")

plot_lines_Median_IQR(df.dropna(subset=["finHarvDat"]))

## normalize using max value in each 3week period (aka image)


In [8]:
def get_top_N_features(N:int, COLUMN_NAMES:list([str]), df_trimmed: pd.DataFrame):
    dics = []
    for i in range(0,4):
        dataframes_dic = plots.plot_per_period(utilities.get_classes_colors(COLUMN_NAMES), df_trimmed,
                                            COLUMN_NAMES, f"../plots/bar_plots/seperability{i}.pdf", metric=i)
        dics.append(dataframes_dic)
    
    df_scores = pd.DataFrame({"class" : np.array(COLUMN_NAMES), 
                                "score": np.zeros(len(COLUMN_NAMES))}).set_index('class')

    for i in range(4): # for each separability metric
        curr_dic = dics[i]
        for image_idx in sorted(dics[0].keys()): # [i7 i8 i9]
            curr_df = curr_dic[image_idx]
            curr_df['score'] = np.array(curr_df['value'][:])
            
            if(i >= 2): # because the two other separability metrics results say that they are separable
                            #   when the result is lower, opposite to the first two where the result is more 
                            #  separable when the value is higher
                curr_df['score'] = 1/curr_df['score']
                
            curr_df.sort_values(by=['score'], ascending=False, inplace=True)
            
            # normalize based on the max value
            curr_df['score'] = curr_df['score']/np.max(curr_df['score'])
            
            joined_df = df_scores.join(curr_df.set_index("class"), how="left", rsuffix="_right")
            joined_df['score_right'] = joined_df['score_right'].fillna(0) # replace nans with zeros
            df_scores['score'] = df_scores['score'] + joined_df['score_right']
        
        
    df_scores.sort_values(by=['score'],  ascending=False, inplace=True)
    TOP_FEATURES = list(df_scores[:N].index)
    
    return df_scores

res = get_top_N_features(15, NUMERIC_COLS, df_trimmed)
# from these indices, some of them can be derived from planet satalite, which is great
# these findings can be used to improve the prediction that is already in use using planet satalite

In [9]:
res.to_csv("top_feats.csv")

In [10]:
res.T.columns

Index(['NDVI', 'RVI', 'BSI', 'MCARI_prev', 'B4', 'RVI_prev', 'GNDVI', 'NDWI',
       'ARVI', 'B12', 'NDVI_prev', 'B3', 'NBRI', 'MSI_prev', 'NDI45', 'B2',
       'PSSRa', 'NDVIre', 'NBRI_prev', 'B11', 'GCI', 'NDVIre_prev', 'EVI_prev',
       'GNDVI_prev', 'NDWI_prev', 'NIRv_diff', 'EVI', 'NDMI_prev', 'B5',
       'PSSRa_prev', 'GCI_prev', 'MSI', 'NDTI', 'MTCI_prev', 'CIr', 'BSI_prev',
       'NDMI', 'GCI_diff', 'NDTI_diff', 'NDI45_prev', 'MCARI', 'CIr_prev',
       'IRECI_prev', 'CIr_diff', 'MSI_diff', 'MTCI', 'IRECI', 'B6',
       'NDTI_prev', 'NDSI_prev', 'GNDVI_diff', 'NDWI_diff', 'NIRv', 'B8',
       'ARVI_prev', 'NDSI', 'PSSRa_diff', 'MTCI_diff', 'NBRI_diff',
       'NDI45_diff', 'RVI_diff', 'IRECI_diff', 'NIRv_prev', 'NDMI_diff',
       'NDSI_diff', 'MCARI_diff', 'BSI_diff', 'EVI_diff', 'NDVI_diff', 'B7',
       'B8A', 'NDVIre_diff', 'ARVI_diff'],
      dtype='object', name='class')

In [11]:
res.head()

,score
class,
NDVI,7.100975
RVI,7.074619
BSI,7.017032
MCARI_prev,6.307646
B4,6.104225


In [ ]:
['NDVI', 'RVI', 'BSI', 'MCARI_prev', 'B4']

In [ ]:
# recommend to use indices with B11 and/or B12 for sentinel-based prediction

# recommend to use indices with lower Bs for planet-based prediction

In [ ]:
    """
    We need to investigate if using sentinel-2 based indices &
    bands are better performing compared to those based on bands dependent on planet satalite
    
    
    """

In [ ]:
"""
background (3 slides)

purpose:



conclusions:
planet data recommendations
sentinel-2 data recommendations


last slide:

mention potentional effects on future applications

# mention future potential improvements:
- using a pretrained model & fine tune the last layers for our remote sensing purposes
- have multiple models for differnt parts of the year. Our scoring approach takes in consideration 
            the performance across the harvest season, but we found that some bands & indices 
                    perform better at differnt parts of the hervest season: for example, B12 
                        is really good later in the summer(August), but not that good in other months.

"""
    

In [ ]:
#planet bands: RGB & NIR